###A basic YOLO architecture:

A simplified YOLO model with the following:
1. Convolutional backbone
2. Feature extraction
3. Classification head

###Basic Convolution block:
Conv2D -> BatchNorm -> ReLU

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple, List, Dict, Optional

class ConvBlock(nn.Module):
    """
    Conv2d -> BatchNorm -> ReLU

    Args:
        in_channels (int): Number of input channels
        out_channels (int): Number of output channels
        kernel_size (int): Size of the convolutional kernel
        stride (int): Stride of the convolution
    """
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 3,
        stride: int = 1
    ):
        super().__init__()
        padding = kernel_size // 2

        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv(x)

###Basic YOLO architecture

Model Structure:

1.   Feature Extraction:
        *   Stage 1: 416x416 -> 208x208 (32 channels)
        *   Stage 2: 208x208 -> 104x104 (64 channels)
        *   Stage 3: 104x104 -> 52x52 (128 channels)
        *   Stage 4: 52x52 -> 26x26 (256 channels)

2.   Classification Head:

        *   Global Average Pooling
        *   Fully Connected Layers


In [23]:
class BasicYOLO(nn.Module):
    """Basic YOLO architecture for classification.

    Args:
        num_classes (int): Number of output classes
        input_shape (Tuple[int, int, int]): Input shape (channels, height, width)
        class_names (List[str], optional): Names of the classes
    """
    def __init__(
        self,
        num_classes: int,
        input_shape: Tuple[int, int, int],
        class_names: Optional[List[str]] = None
    ):
        super().__init__()

        # Validate input shape
        channels, height, width = input_shape
        assert channels == 3, f"Expected 3 input channels, got {channels}"
        assert height == width, f"Expected square input, got height={height}, width={width}"

        # Initialize protected attributes directly
        self._input_shape = input_shape
        self._num_classes = num_classes
        self._class_names = class_names if class_names else [f"class_{i}" for i in range(num_classes)]

        # Backbone network - 4 stages of convolution blocks
        self.features = nn.Sequential(
            # Stage 1: 416x416 -> 208x208
            ConvBlock(channels, 32, stride=2),
            ConvBlock(32, 32),

            # Stage 2: 208x208 -> 104x104
            ConvBlock(32, 64, stride=2),
            ConvBlock(64, 64),

            # Stage 3: 104x104 -> 52x52
            ConvBlock(64, 128, stride=2),
            ConvBlock(128, 128),
            ConvBlock(128, 128),

            # Stage 4: 52x52 -> 26x26
            ConvBlock(128, 256, stride=2),
            ConvBlock(256, 256),
            ConvBlock(256, 256)
        )

        # Classification head
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, channels, height, width)

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, num_classes)
        """
        x = self.features(x)
        x = self.classifier(x)
        return x

    def predict(self, x: torch.Tensor, threshold: float = 0.5) -> List[Dict[str, float]]:
        """Predict class probabilities and labels for input images.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, channels, height, width)
            threshold (float, optional): Confidence threshold for predictions. Defaults to 0.5.

        Returns:
            List[Dict[str, float]]: List of dictionaries containing class probabilities
        """
        self.eval()  # Set to evaluation mode
        with torch.no_grad():
            # Get model predictions
            logits = self(x)
            probs = F.softmax(logits, dim=1)

            # Convert to numpy for easier processing
            probs_np = probs.cpu().numpy()

            # Process each image's predictions
            batch_predictions = []
            for image_probs in probs_np:
                # Get predictions above threshold
                class_predictions = {}
                for class_idx, prob in enumerate(image_probs):
                    if prob >= threshold:
                        class_predictions[self._class_names[class_idx]] = float(prob)

                # If no class meets threshold, return highest probability class
                if not class_predictions:
                    max_idx = image_probs.argmax()
                    class_predictions[self._class_names[max_idx]] = float(image_probs[max_idx])

                batch_predictions.append(class_predictions)

        return batch_predictions

    def get_input_shape(self) -> Tuple[int, int, int]:
        """Get expected input shape."""
        return self._input_shape

    def get_num_classes(self) -> int:
        """Get number of output classes."""
        return self._num_classes

    def get_class_names(self) -> List[str]:
        """Get list of class names."""
        return self._class_names

    def set_class_names(self, class_names: List[str]):
        """Set class names.

        Args:
            class_names (List[str]): List of class names
        """
        assert len(class_names) == self._num_classes, \
            f"Expected {self._num_classes} class names, got {len(class_names)}"
        self._class_names = class_names

In [24]:
def create_model(
    num_classes: int,
    input_shape: Tuple[int, int, int] = (3, 416, 416),
    class_names: Optional[List[str]] = None
) -> BasicYOLO:
    """Creates and returns a YOLO model.

    Args:
        num_classes (int): Number of classes to predict
        input_shape (Tuple[int, int, int]): Input shape as (channels, height, width)
        class_names (List[str], optional): Names of the classes. Defaults to None.

    Returns:
        BasicYOLO: Initialized YOLO model
    """
    return BasicYOLO(
        num_classes=num_classes,
        input_shape=input_shape,
        class_names=class_names
    )

Example usage

In [25]:
if __name__ == "__main__":
    # Test parameters
    INPUT_SHAPE = (3, 416, 416)
    NUM_CLASSES = 3
    BATCH_SIZE = 2
    CLASS_NAMES = ["cat", "dog", "bird"]

    # Create model
    model = create_model(
        num_classes=NUM_CLASSES,
        input_shape=INPUT_SHAPE,
        class_names=CLASS_NAMES
    )

    # Print model info
    print(f"Model Configuration:")
    print(f"Input shape: {model.get_input_shape()}")
    print(f"Number of classes: {model.get_num_classes()}")
    print(f"Class names: {model.get_class_names()}")

    # Test forward pass with sample data
    x = torch.randn(BATCH_SIZE, *INPUT_SHAPE)

    # Get predictions with class labels
    predictions = model.predict(x, threshold=0.3)

    # Print results
    print("\nTest Predictions:")
    for i, pred in enumerate(predictions):
        print(f"Image {i + 1} predictions:")
        for class_name, prob in pred.items():
            print(f"  {class_name}: {prob:.3f}")

Model Configuration:
Input shape: (3, 416, 416)
Number of classes: 3
Class names: ['cat', 'dog', 'bird']

Test Predictions:
Image 1 predictions:
  cat: 0.359
  dog: 0.311
  bird: 0.330
Image 2 predictions:
  cat: 0.359
  dog: 0.311
  bird: 0.330
